## Project Scope: Delta Hedging Simulation

This project simulates the profit and loss (P&L) of delta-hedging a European call option using a self-financing trading strategy over discrete time steps.

### Initial Parameters

- **Spot Price** ($S_0$): $50  
- **Strike Price** ($K$): $50  
- **Time to Maturity** ($T$): 0.25 years (3 months)  
- **Volatility** ($\sigma$): 30%  
- **Risk-Free Interest Rate** ($r$): 2%  
- **Number of Options**: 100,000  
- **Number of Hedging Periods**: 50

### Goal

To evaluate the hedging performance (hedging P&L and total P&L) of a dynamically rebalanced, self-financing portfolio that attempts to replicate the payoff of a short or long European call option over time.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

S0 = 50
T = 0.25
sigma = 0.3
r = 0.02
K = 50
num_options = 100000
num_hedging_periods = 50

In [2]:
df = pd.read_csv('data/StockPath.csv')
df.head()

,Period,Stock0,Stock1,Stock2,Stock3,Stock4
0,0,50.000000,50.000000,50.000000,50.000000,50.000000
1,1,49.507226,49.672466,49.177203,48.669910,47.640628
2,2,48.005821,49.625614,49.818076,46.169337,49.449656
3,3,47.765455,49.743893,51.179055,47.481015,49.372705
4,4,47.946162,50.215642,50.332767,49.034295,49.947589


In [15]:
def compute_log_return(prices, num_periods, num_periods_per_year):
    
    log_returns = np.log(prices[1:] / prices[:-1])
    mean_returns = np.sum(log_returns)/num_periods
    
    annualized_mean_returns = mean_returns * num_periods_per_year
    std_per_period = np.std(log_returns, ddof=1)  # use ddof=1 for sample std dev
    
    annualized_std = std_per_period * np.sqrt(num_periods_per_year)
    print(f'Avg return (annualised): {round(annualized_mean_returns, 4)}; Std return (annualised): {round(annualized_std, 4)}')
    
    return log_returns, annualized_mean_returns, annualized_std

print(f'\nStock 0')
log_returns, annualized_mean_returns, annualized_std = compute_log_return(df['Stock0'].to_numpy(), num_hedging_periods, num_hedging_periods/T)
print(f'\nStock 1')
log_returns, annualized_mean_returns, annualized_std = compute_log_return(df['Stock1'].to_numpy(), num_hedging_periods, num_hedging_periods/T)
print(f'\nStock 2')
log_returns, annualized_mean_returns, annualized_std = compute_log_return(df['Stock2'].to_numpy(), num_hedging_periods, num_hedging_periods/T)
print(f'\nStock 3')
log_returns, annualized_mean_returns, annualized_std = compute_log_return(df['Stock3'].to_numpy(), num_hedging_periods, num_hedging_periods/T)
print(f'\nStock 4')
log_returns, annualized_mean_returns, annualized_std = compute_log_return(df['Stock4'].to_numpy(), num_hedging_periods, num_hedging_periods/T)
    


Stock 0
Avg return (annualised): 0.3379; Std return (annualised): 0.249

Stock 1
Avg return (annualised): -0.1459; Std return (annualised): 0.2542

Stock 2
Avg return (annualised): 0.3488; Std return (annualised): 0.3036

Stock 3
Avg return (annualised): 0.9897; Std return (annualised): 0.5157

Stock 4
Avg return (annualised): -0.6034; Std return (annualised): 0.4014


## Self-Financing Delta-Hedging Portfolio Update Formula

The portfolio value at the next time step, $i+1$, is given by:

$$
V_{i+1} = V_i + \delta_i (S_{i+1} + S_i c \Delta t - S_i) + (V_i - \delta_i S_i)(e^{r \Delta t} - 1)
$$

### Explanation of Terms:
- $V_i$: Portfolio value at time step $i$
- $V_{i+1}$: Portfolio value at time $i+1$
- $\delta_i$: Option delta at time $i$
- $S_i$: Stock price at time $i$
- $S_{i+1}$: Stock price at time $i+1$
- $c$: Continuous dividend yield
- $r$: Risk-free interest rate
- $\Delta t$: Time step size (e.g., $1/252$ for daily hedging)

### Intuition:
- The first term  
  $\delta_i (S_{i+1} + S_i c \Delta t - S_i)$  
  reflects gains/losses from the **stock position**, adjusted for dividends.

- The second term  
  $(V_i - \delta_i S_i)(e^{r \Delta t} - 1)$  
  reflects the **interest earned** on the cash account (non-stock part of portfolio).

Together, these components update the value of a **self-financing** delta-hedging portfolio that replicates the option value over time without external funding.


In [49]:
def delta_hedging_portfolio(S, delta, V0, r, c, dt):
    """
    Calculate self-financing delta hedging portfolio value over time.

    Parameters:
    S : np.ndarray        # stock prices (length n+1)
    delta : np.ndarray    # delta values (length n)
    V0 : float            # initial portfolio value
    r : float             # risk-free rate
    c : float             # dividend yield
    dt : float            # time step size

    Returns:
    V : np.ndarray        # portfolio values (length n+1)
    """
    n = len(delta)
    V = np.zeros(n + 1)
    V[0] = V0
    
    for i in range(n):
        stock_term = delta[i] * (S[i+1] + S[i] * c * dt - S[i])
        cash_term = (V[i] - delta[i] * S[i]) * (np.exp(r * dt) - 1)
        V[i+1] = V[i] + stock_term + cash_term

    return V

# Black-Scholes formula for option pricing
def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return option_price, d1, d2

def delta(S, K, T, r, sigma, option_type='call'):
    __, d1, _ = black_scholes(S, K, T, r, sigma, option_type)
    if option_type == 'call':
        return norm.cdf(d1)
    elif option_type == 'put':
        return norm.cdf(d1) - 1
    
    


## Hedging P&L in Delta Hedging Strategy

In a delta-hedging framework, the **hedging P&L** measures how well your dynamic, self-financing portfolio replicates the option's payoff at maturity.

### General Formula:

For an option with payoff $\Pi_T$ at maturity $T$, and a hedging portfolio value $V_T$ at maturity:

$$
\text{Hedging P\&L} = V_T - \Pi_T
$$

- $V_T$: Final value of the delta-hedging portfolio (stock + cash)
- $\Pi_T$: Payoff of the option at expiry (e.g., for a call, $\max(S_T - K, 0)$)

---

### Interpretation Based on Position

- If you are **short** the option:
  - You sold it and tried to hedge its payoff.
  - **You want:** $V_T$ to match $\Pi_T$, so you can deliver the payoff without loss.
  - **Hedging P&L = final portfolio – what you owe**

- If you are **long** the option:
  - You bought the option and tried to replicate it yourself.
  - **You want:** $V_T$ to be **less than** $\Pi_T$, meaning your hedge cost less than the payout.
  - **Hedging P&L = what you receive – what your hedge cost**

---

### Ideal Case:
Under **perfect hedging** (continuous time, no transaction costs, exact model parameters):

$$
V_T = \Pi_T \quad \Rightarrow \quad \text{Hedging P\&L} = 0
$$

But in reality:
- Hedging is done in **discrete steps**
- There's **model error**, **volatility changes**, and **transaction costs**

So the hedging P&L is **typically nonzero**, and reflects **hedging performance**.

---

### Total P&L in Delta Hedging Strategy

While **hedging P&L** measures how well your delta-hedging portfolio replicates the option payoff, the **total P&L** reflects your **actual profit or loss from trading the option** including the initial premium and the cost of hedging.

---

### Formula for Total P&L

If you are **short** the option and sell it for a premium at time $0$:

$$
\text{Total P\&L} = \text{Premium Received} + V_T - \Pi_T
$$

If you are **long** the option and buy it for a premium:

$$
\text{Total P\&L} = \Pi_T - \text{Premium Paid} - V_T
$$

Where:
- $V_T$: Final value of the hedging portfolio
- $\Pi_T$: Payoff of the option at maturity
- $\text{Premium}$: Price paid or received for the option at time $0$

In [52]:

for stock in ['Stock0', 'Stock1', 'Stock2', 'Stock3', 'Stock4']:
    S_lst = np.array(df[stock])
    T_lst = np.array(T - 1/num_hedging_periods * df['Period'] * T)
    delta_lst = []
    call, d1, d2 = black_scholes(S0, K, T, r, sigma, option_type='call') 
    V0 = call * num_options # Initial portfolio value
    for i in range(len(S_lst)-1):
        delta_lst.append(delta(S_lst[i], K, T_lst[i], r, sigma, 'call'))
    delta_lst = np.array(delta_lst) * num_options # Delta determines the amount of stocks to hedge
        
    V_lst = delta_hedging_portfolio(S = S_lst, delta = delta_lst, V0 = V0, r = r, c = 0, dt = T/num_hedging_periods)
    hedging_pnl = V_lst[-1] - max(S_lst[-1] - K, 0) * num_options
    print(f'For {stock}, the hedging P&L is {round(hedging_pnl, 2)}\n')


For Stock0, the hedging P&L is 55393.58

For Stock1, the hedging P&L is 46624.88

For Stock2, the hedging P&L is -20263.07

For Stock3, the hedging P&L is -122866.99

For Stock4, the hedging P&L is -104400.91

